Check if scraping is allowed using robots.txt

In [ ]:
from urllib import robotparser

robots_url = "https://en.wikipedia.org/robots.txt"
rp = robotparser.RobotFileParser()
rp.set_url(robots_url)
rp.read()

In [ ]:
target = "https://en.wikipedia.org/wiki/Artificial_intelligence"
print("Allowed to scrape?", rp.can_fetch("*", target))

Sending an HTTP GET request

In [ ]:
import requests

In [ ]:
url = "https://en.wikipedia.org/wiki/Artificial_intelligence"
headers = None

resp = requests.get(url, headers=headers, timeout=5)

In [ ]:
print(f"Status code : {resp.status_code}")
print(f"Content-Type : {resp.headers.get('Content-Type')}")
print(f"First 100 char of HTML page : {resp.text[:100]}")

Parsing HTML with BeautifulSoup

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(resp.text, "lxml")
# soup = BeautifulSoup(resp.text, "html.parser")

In [ ]:
soup.title.get_text()

In [ ]:
heading = soup.find("h1", id="firstHeading").get_text(strip=True)
print("Main heading : ", heading)

In [ ]:
content_div = soup.find("div", id="mw-content-text")

first_paragraph = ""

for p in content_div.find_all("p"):
    text = p.get_text(strip=True)
    if len(text) > 50:
        first_paragraph = text
        break

print(f"First paragraph : {first_paragraph}")

In [ ]:
from urllib.parse import urljoin

base = "https://en.wikipedia.org"

internal_links = set()

for a in soup.select("#mw-content-text a[href^='/wiki/']"):
    href = a.get('href')
    if href and ':' not in href:
        full = urljoin(base, href)
        internal_links.add(full)

print("Internal wiki links found: ", len(internal_links))
print("Sample links: ")
for link in internal_links:
    print(link)

In [ ]:
import csv
out_file = "ai_wikipedia_summary.csv"

with open(out_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["field", "value"])
    writer.writerow(["heading", heading])
    writer.writerow(["first_paragraph", first_paragraph])
    writer.writerow(["top_internal_links_count", len(internal_links)])

print("Saved summary to", out_file)